In [11]:
from minizinc import Instance, Model, Solver

In [13]:
%load_ext iminizinc

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [21]:
from minizinc import Instance, Model, Solver

# Load n-Queens model from file
nqueens = Model("./adv_math_modelling_proj/resource 4.mzn")
# Find the MiniZinc solver configuration for Gecode
gecode = Solver.lookup("gecode")
# Create an Instance of the n-Queens model for Gecode
instance = Instance(gecode, nqueens)
# Assign 4 to n
instance["n"] = 4
result = instance.solve()
# Output the array q
print(result["q"])

/usr/lib/python3.10/contextlib.py:539: RuntimeWarning: coroutine 'Instance.solve_async' was never awaited
  def _fix_exception_context(new_exc, old_exc):


TypeError: /usr/share/minizinc/std//nvalue_fn.mzn:7.57-67:
MiniZinc: type error: no function or predicate with this signature found: `nvalue(var int,array[$_] of var int)'
Cannot use the following functions or predicates with the same identifier:
function var int : nvalue(array[$_] of var int: x);
    (requires 1 argument, but 2 given)
predicate nvalue(var int: n,array[int] of var int: x);
    (argument 2 expects type array[int] of var int, but type array[$_] of var int given)
File fragment:
6: function var int: nvalue(array[$X] of var int: x) =
7:   let { var 0..length(x): n::is_defined_var; constraint nvalue(n,x)::defines_var(n); } in n;
                                                           ^^^^^^^^^^^


In [16]:
%%minizinc -a

int: n = 5; % The number of queens.

array [1..n] of var 1..n: q;

include "alldifferent.mzn";

constraint alldifferent(q);
constraint alldifferent(i in 1..n)(q[i] + i);
constraint alldifferent(i in 1..n)(q[i] - i);

[{'q': [4, 2, 5, 3, 1]},
 {'q': [3, 5, 2, 4, 1]},
 {'q': [5, 3, 1, 4, 2]},
 {'q': [4, 1, 3, 5, 2]},
 {'q': [1, 4, 2, 5, 3]},
 {'q': [5, 2, 4, 1, 3]},
 {'q': [2, 5, 3, 1, 4]},
 {'q': [1, 3, 5, 2, 4]},
 {'q': [2, 4, 1, 3, 5]},
 {'q': [3, 1, 4, 2, 5]}]

In [20]:
%%minizinc

% Есть 15 изделий, имеющих срок службы и ресурс до кап. ремонта.
% Дата изготовления изделий различается в пределах года.
% Ремонт длится 1 год.
% Можно превысить срок службы на два года, а ресурс на 200 часов.
% Необходимо распределить даты отправки в ремонт так, чтобы в эксплуатации оставалось 5 (или максимум) единиц.

include "globals.mzn";

%%% Initial data

int: res = 750;
int: srok = 120;

int: res_max = 1000;

int: res_incr = 20; % operation time increment per month

int: n = 15;
set of int: dev = 1..n;

array[dev] of int: op_time = [300, 250, 350, 450, 650, 670, 660, 220, 250, 670, 190, 560, 340, 370, 360]; % at the moment when 10 years expire

array[dev] of int: prod_date = [11, 0, 14, 0, 0, 2, 2, 2, 4, 4, 4, 5, 9, 9, 14]; 

set of int: month = 120..164;


%%% Declare variables

var int: srok_max;

array[dev] of var int: rem_time;

var int: stay;

array[dev] of var int: rem_date;

array[dev] of var int: return_date;

array[dev] of var int: rem_res;

array[dev] of var int: res_penalty;

array[dev] of var int: srok_penalty;

var int: stay_penalty;

var int: final_penalty;

array[dev] of var int: rem_time_penalty;

var int: fin_fin_penal;

array[month] of var int: staying;



%%% Declare weights of change

int: res_incr_price = 1; % per 1 h of res increase
int: rem_time_decr = 2000; % per 1 month of repair of time decrease
int: srok_max_incr = 200; % per 1 month of expl time incr
int: stay_decr = 4000; % per 1 machine staying decrease
int: res_unused = 3; % penalty for unused 1h of op_time



%%% Constraints

constraint
srok_max >= 144 /\ srok_max <= 152;

constraint
forall(i in dev)(rem_time[i] <= 14 /\ rem_time[i] >= 10);

constraint
stay >= 4 /\ stay <= n;

constraint
forall(i in dev)(rem_date[i] <= srok_max + prod_date[i] /\ rem_date[i] >= srok  + prod_date[i] - 3);

constraint
forall(i in rem_date, j in op_time)((i - srok)*res_incr + j <= res_max);

constraint
forall(i in dev)(return_date[i] = rem_date[i] + rem_time[i]);


% constraint
% forall(i in index_set(month))(staying[i] = card({j | j in dev where month[i] <= rem_date[j] \/ month[i] >= return_date[j]}));

% constraint
% forall(i in index_set(month))(staying[i] >= stay);

% constraint
% forall(i in month)(card({j | j in dev where i <= rem_date[j] \/ i >= return_date[j]}) >= stay);

constraint
forall(i in month)(staying[i] = card({j | j in index_set(rem_date) where i < rem_date[j] \/ i > rem_date[j] + rem_time[j]}));

constraint
forall(i in month)(staying[i] >= stay);

constraint
forall(i in dev)(rem_res[i] = op_time[i] + (rem_date[i] - srok)*res_incr);

constraint
forall(i in dev)(if rem_date[i] >= prod_date[i] + srok then srok_penalty[i] = (rem_date[i]-prod_date[i] - srok)*srok_max_incr else srok_penalty[i] = 0 endif);

constraint
forall(i in dev)(if rem_res[i] >= 750 then res_penalty[i] = (rem_res[i]-res)*res_incr_price else res_penalty[i] = (res - rem_res[i])*res_unused endif);

% constraint
% if stay < 5 then stay_penalty = (5 - stay)*stay_decr else stay_penalty = 0 endif;

constraint
stay_penalty = card({i | i in month where staying[i] < 5}) * stay_decr;


% constraint
% forall(i in dev)(if rem_time[i] < 12 then rem_time_penalty[i] = rem_time_decr * (12-rem_time[i]) else rem_time_penalty[i] = 0 endif);

constraint
forall(i in dev)(if rem_time[i] < 12 then rem_time_penalty[i] = (12 - rem_time[i])*rem_time_decr else rem_time_penalty[i] = 0 endif);

constraint
final_penalty = sum(i in dev)(srok_penalty[i]+res_penalty[i]+rem_time_penalty[i]);


constraint
fin_fin_penal = final_penalty + stay_penalty;



%%% Solve

solve
%   :: warm_start_array( [                     %%% Can be on the upper level
%      warm_start( rem_date, [132,120,133,120,120,122,122,122,129,128,136,129,131,131,133] ),               %%% Use <> for missing values
%      warm_start( rem_time, [8,8,8,8,8,8,8,8,8,8,8,8,8,8,8] ),
% %      warm_start( stay, 3 ),
%                         ])
%    :: int_search(rem_date, dom_w_deg, indomain_median)
    minimize fin_fin_penal;


%%% Output config 

% output [
%   show(fix(fin_fin_penal))] ++ [show(fix(rem_date))] ++ [show(fix(rem_time))] ++ [

%   if j >= fix(rem_date[i]) /\ j <= fix(return_date[i]) then "#" else "|" endif 
%   | i in dev, j in month

% ] ++ ["\n"]

  

/usr/share/minizinc/std//nvalue_fn.mzn:7.57-67:
MiniZinc: type error: no function or predicate with this signature found: `nvalue(var int,array[$_] of var int)'
Cannot use the following functions or predicates with the same identifier:
function var int : nvalue(array[$_] of var int: x);
    (requires 1 argument, but 2 given)
predicate nvalue(var int: n,array[int] of var int: x);
    (argument 2 expects type array[int] of var int, but type array[$_] of var int given)
